In [ ]:
p

In [ ]:
%load_ext autoreload
%autoreload 2

from os.path import join as pjoin    

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


ipy = get_ipython()
p = pjoin(scripts_dir,'read_MEG_cc.py')
ipy.run_line_magic('run',f'-i {p}')

def mplresestcol():
    COLOR = 'black'
    COLOR_INV = 'white'
    mpl.rcParams['text.color'] = COLOR
    mpl.rcParams['axes.labelcolor'] = COLOR
    mpl.rcParams['xtick.color'] = COLOR
    mpl.rcParams['ytick.color'] = COLOR
    mpl.rcParams["figure.facecolor"] = COLOR_INV
    mpl.rcParams["axes.facecolor"] = COLOR_INV
mplresestcol()

In [ ]:
chn_diode = 'UADC001-2800'
#chn_pupil_size = 'UADC012-2800'
chn_time = 'SCLK01-177'

In [ ]:
subjects[-2:]

In [ ]:
subj2all = {}
# 30 had diode not working for first 30-ish trials

In [ ]:
for subj in subjects:
    del subj2all[subj]['rs']
#del subj2all[subj]['rs']

In [ ]:
# do
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
from meg_proc import loadBehavOneSubj
from meg_proc import events2df
from behav_proc import aggRows

replace = 0
#replace_rs = 1

#for subj in subjects[-2:-1]:#[:2]:
#for subj in ['2023-SE2-016','2023-SE2-018']:
#for subj in ['2023-SE2-023']:
#for subj in ['2023-SE2-002']:
#for subj in ['2023-SE2-010']:
for subj in ['2023-SE2-030']:
#for subj in ['2023-SE2-016']:
#for subj in ['2023-SE2-019']:
#for subj in ['2023-SE2-018']:
#for subj in subjects:
    print(f'-----------  Start subject = {subj} ---------')
    if replace:
        subj2all[subj] = {}
    else:
        for k in ['loc_events2df','lf','loc_events2df','rs']:
            if k in subj2all[subj]:
                del subj2all[subj][k]
    path_behav = pjoin( subj2path[subj] , 'behav')
        
    if 'rs' not in subj2all[subj]:
        try:
            if subj in ['2023-SE2-016','2023-SE2-018','2023-SE2-019','2023-SE2-030']:
                flag = 'read_all'
            else:
                flag = 'read_last'
            rs = loadBehavOneSubj(path_behav, flag)
            
            df,dftriglog, df_trialinfoseq_params, \
                params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT = rs[-1]
            
            for i in range(len(rs)):
                print('RS content:  ',i, rs[i][0]['trial_index'].max() )

            if flag == 'read_all':
                dfs = [rs[i][0] for i in range(len(rs))]
                dfa = concatBehavDfs(dfs)
                df= dfa
            
        except Exception as e:
            excsfmt = traceback.format_exc()
            print(excsfmt)

            subj2all[subj]['excsfmt_readbehav'] = excsfmt
            continue
    
        m1,m2 = df_trialinfoseq_params['trial_index'].max(),df['trial_index'].max()
        if m1 != m2:
            print(f'WARNING: Experiment should have had {m1} trials, but the log only found {m2}')

        subj2all[subj]['rs'] = rs
        subj2all[subj]['df'] = df
    else:
        rs  = subj2all[subj]['rs']
        df,dftriglog, df_trialinfoseq_params, \
                params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT = rs[-1]
        
        if flag == 'read_all':
            dfs = [rs[i][0] for i in range(len(rs))]
            dfa = concatBehavDfs(dfs)
            df= dfa
        subj2all[subj]['rs'] = rs
        subj2all[subj]['df'] = df
        

    
    from meg_proc import read_raw_MEG_CC
    fnfs_ds = subj2dsnames[subj]
    if 'raw' not in subj2all[subj]:
        raw,events = read_raw_MEG_CC(fnfs_ds, fewch=True, verbose=0)
    else:
        raw  = subj2all[subj]['raw']
        events = mne.find_events(raw, shortest_event=2)
    raw.load_data()
    subj2all[subj]['raw'] = raw
    
    restmintime = df.query('phase == "REST"')['time_abs'].min()
    #df['time_MEG_aligned'] = df['time_abs'] + (restmintime_meg - retime_absstmintime)
    df['time_since_first_REST'] = df['time_abs']  - restmintime

    dat_time ,times = raw[chn_time] 
    diode,times = raw[chn_diode]
    
    subj2all[subj]['dat_time'] = dat_time
    
    #print(subj)
    import sys,traceback
    try:
        #dat_time
        dfev, loc =events2df(events, raw, diode, dat_time,
                    trigger2phase, CONTEXT_TRIGGER_DICT, params, restmintime,
                         return_aux = 1)
        subj2all[subj]['loc_events2df'] = loc
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)
        ei = exc_info[2]    
        #display(ei.tb_frame)
        psf = ei

        lfprev = None; lf = None
        lfs = []
        di = 0
        while psf.tb_frame is not None:
            stackframe = psf.tb_frame        

            psf = psf.tb_next  
            if ('conda' not in stackframe.f_code.co_filename) and\
                ('mamba' not in stackframe.f_code.co_filename):        
                display(di, stackframe)
                lfprev = lf
                lf = stackframe.f_locals
                lfs += [lf]
                di += 1
            if psf is None:
                break

        if 'dfev' in lf:
            dfev = lf['dfev']
            subj2all[subj]['lf'] = lf                   
        else:
            print('--#######--- dfev NOT SET')
        excsfmt = traceback.format_exc()
        print(excsfmt)
        
        subj2all[subj]['excsfmt'] = excsfmt
        
    subj2all[subj]['dfev'] = dfev


    m1,m2 = dfev['trial_index'].max(), df['trial_index'].max()
    if m1 != m2:
        print(f'Trial numbers inconsistent: dfev = {m1}, df = {m2}')
    # 006 if when MEG broke after prelast block
    cond = (m2 == m1 + 1) and \
        set(df.query('trial_index == 734').phase.unique()) == {'REST','BREAK'}
    if subj == '2023-SE2-006' and cond:
        print('But iconsistency is okay')
    #else:
    #    assert m1 == m2, (m1,m2)

    c = df['phase'] != df['phase'].shift(1)
    c[0] = False
    df['phase_ind'] = np.cumsum(c)
    # it is important to take min and not max here because triggers arrive in the beg of phase, not in the end
    dfph = aggRows(df, 'time_abs', 'min', colgrp = 'phase_ind')
    dfph['time_since_prev_phase'] = dfph['time_since_first_REST'].diff()    
    subj2all[subj]['dfph'] = dfph

    import gc; gc.collect()

    print('----------------------------------')

In [ ]:
subj 

In [ ]:
def concatBehavDfs(dfs):
    dfs_ = []
    last_tr = 0
    for df in dfs:
        df = df.copy()
        df['trial_index'] += last_tr
        dfs_ += [df]
        last_tr = df.trial_index.max() + 1
    dfa = pd.concat(dfs_,ignore_index=1)
    lb = len( dfa[dfa['time_abs'].diff() < 0] ); print(lb)
    assert lb == 0
    
    return dfa
df = concatBehavDfs(dfs)
df.trial_index.max()

In [ ]:
df[df['time_abs'].diff() < 0]

In [ ]:
# 30 restart from last trial

In [ ]:
df[df['time'].diff() < 0]

## trial corresp

### with diodes

In [ ]:
import gc; gc.collect()

In [ ]:
#dfev['time_megchan'].plot() 
#dfev['time_since_first_REST'].plot() 
dfev.plot(x='nsample',y=['time_since_first_REST','time_since_first_REST_diode'])
#dfev.plot(x='nsample',y=['time_since_first_REST','time_since_first_REST_diode'])
#dfev.plot(x='nsample',y='time_since_first_REST_diode')

df_diode = loc['df_diode']
#df_diode['time_since_first_REST0'].plot(x=df_diode['nsample'])
df_diode.plot(x='nsample',y=['time_since_first_REST','time_since_first_REST0'])

df_diode_clean = loc['df_diode_clean']
#df_diode['time_since_first_REST0'].plot(x=df_diode['nsample'])
df_diode_clean.plot(x='nsample',y=['time_since_first_REST','time_since_first_REST0'])

In [ ]:
#dfph.plot?

In [ ]:
ax = plt.gca()
dfph.plot(x='phase_ind',y=['time_since_first_REST'], ax = ax)
dfev.plot(x='phase_ind',y=['time_since_first_REST','time_since_first_REST_diode'], ax = ax)

In [ ]:
#Trial numbers inconsistent: dfev = 859, df = 860

In [ ]:
dfph.query('phase == "REST"')[cols]

In [ ]:
(dfev.query('~time_since_first_REST_diode.isna()')['time_since_first_REST_diode'].diff() .iloc[1:] <= 0).sum()

In [ ]:
dfev.query('~time_since_first_REST_diode.isna()').nsample.max()

In [ ]:
dfev.query('~time_since_first_REST_diode.isna()').trial_index.max()

In [ ]:
loc['mvt_starts_megtrig'].nsample.max(), loc['mvt_starts_megtrig_a'].nsample.max()

In [ ]:
dfev.loc[loc['mvt_starts_megtrig'].index].nsample.max()

In [ ]:
df_diode.nsample.max()

In [ ]:
dfev.nsample.max()

In [ ]:
find_correspondence(df)

In [ ]:
df['time_since_first_REST'].max()

In [ ]:
dfev['time_since_first_REST'].max(), dfev['time_megchan'].max()

In [ ]:
dfev['time_since_first_REST_diode'].max()

In [ ]:
df['time_since_first_REST'].plot()

In [ ]:
dfev['time_since_first_REST_diode']

In [ ]:
dfev['time_since_first_REST'].plot()

### trial_numbers

In [ ]:
subj

In [ ]:
df['time_abs'].max()

In [ ]:
ax = plt.gca()
#dfph.query('trial_index <= 170').plot(x='phase_ind',y=['time_abs'], ax = ax)
df.query('trial_index <= 170').plot(x='time_since_first_REST',y=['trial_index'], ax = ax)

In [ ]:
ax = plt.gca()
t = 870
#t = 15
dfph.query('trial_index <= @t').plot(x='time_since_first_REST',y=['phase_ind'], ax = ax)
# event based is red
dfev.query('trial_index <= @t').plot(x='time_since_first_REST',y=['phase_ind'], ax = ax, color='red')

In [ ]:
cols = ['phase','trial_index','time_since_first_REST']
df1 = dfev.loc[loc['ph'].loc[loc['cti']].index]

beh_phprev = dfph.phase.shift(1)
beh_crest = (dfph['phase'] == 'REST') & ( beh_phprev.isin( ['ITI','PAUSE','BREAK','TRAINING_END','MEG_start_trigger']) )

df2 =  dfph.loc[beh_crest,cols]

dfd,dfd_good,dfd_bad,paths = find_correspondence(df1.time_since_first_REST.values,
                                                 df2.time_since_first_REST.values, std_mult =1.5)
best_path = dtw.best_path(paths)
display(dfd_bad)
print( dfd.ind_diff.abs().max() )

In [ ]:
tis = df2.iloc[dfd_bad.ind2.values]
display(tis)
tis = tis.trial_index
#print(tis)
ti = tis.values[0]
print(ti)

In [ ]:
dfdtmp = dfd.query('ind1 != ind2')
display(dfdtmp.iloc[0:3])
tis = df2.iloc[dfdtmp.ind2.values].trial_index
print(tis)
ti = tis.values[0]
print(ti)

In [ ]:
cols = ['phase','trial_index','time_since_first_REST']
df1 = dfev.query('phase == "REST"')

beh_phprev = dfph.phase.shift(1)
beh_crest = (dfph['phase'] == 'REST')# & \
    #( beh_phprev.isin( ['ITI','PAUSE','BREAK','TRAINING_END','MEG_start_trigger']) )

df2 =  dfph.loc[beh_crest,cols]

dfd,dfd_good,dfd_bad,paths = find_correspondence(df1.time_since_first_REST.values,
                                                     df2.time_since_first_REST.values)
best_path = dtw.best_path(paths)

In [ ]:
tis = df2.iloc[dfd_bad.ind2.values].trial_index
print(tis)
ti = tis.values[0]

In [ ]:
from behav_proc import compDf
t1,t2 = ti-2,ti+2
qs = 'trial_index >= @t1 and trial_index <= @t2'
compDf(df1.query(qs),df2.query(qs), ['trial_index','time_since_first_REST'])

In [ ]:
dfd_bad

In [ ]:
len(dfev.query('phase == "REST"')),len(dfph.query('phase == "REST"'))

In [ ]:
# 30 subject has "script restarted in log"

# table 

In [ ]:
dfmax = []
for subj, vs in subj2all.items():
    d = {}
    d['subj']= subj
    
    if 'rs' in vs:
        df,dftriglog, df_trialinfoseq_params, \
            params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT = vs['rs'][-1]
        dfev = vs['dfev']        

        dfevm,dfm = dfev['trial_index'].max(), df['trial_index'].max()
        d['ti_max_beh'] = dfm
        d['ti_max_ev'] = dfevm

        m1 = df_trialinfoseq_params['trial_index'].max()
        d['ti_max_par'] = m1

        if 'lf' in vs:
            k = 'lf'            
            d['len_ts_di']     = len( vs[k]['ts_di']     )
            d['len_ts_di_off'] = len( vs[k]['ts_di_off'] )
            d['err'] = vs.get('excsfmt','\n\n').split('\n')[-2]    
        elif  'loc_events2df' in vs:
            k = 'loc_events2df'
            d['len_ts_di']     = len( vs[k]['ts_di_']     )
            d['len_ts_di_off'] = len( vs[k]['ts_di_off_'] )            
            
            #d['len_ts_di']     = len( vs[k]['ts_di']     )
            #d['len_ts_di_off'] = len( vs[k]['ts_di_off'] )

        
    
    d['err0'] = vs.get('excsfmt_readbehav','\n\n').split('\n')[-2]
    d['timemeg_inc'] = np.all(np.diff(vs['dat_time'] ) [1:] > 0)
    d['timemeg_inc2'] = np.all(np.diff(vs['dat_time'][0,dfev['nsample']] )[1:])
    d['nsample_inc'] = (dfev['nsample'].diff()[1:] > 0).all()
    
    dfmax += [d]
dfmax = pd.DataFrame(dfmax)
dfmax['beh_m_ev'] = dfmax['ti_max_beh'] - dfmax['ti_max_ev']
dfmax['dion_m_off'] = dfmax['len_ts_di'] - dfmax['len_ts_di_off']
dfmax['beh_m_dion'] = dfmax['ti_max_beh'] - dfmax['len_ts_di']
dfmax['ev_m_dion'] = dfmax['ti_max_ev'] - dfmax['len_ts_di']
dfmax

In [ ]:
# 16 Participant made 1 training, we did restart, another training + 1 block 
#  (maybe incomplete), restart from beginning (there was a problem with diode in the     beginning
# 18 had to redo training
# 19 -- nothing in comments, but maybe extra training?
# 30 restarted in the middle of block beacause diode was not working
# 18,19,  30

In [ ]:
subj = '2023-SE2-010'
#vs= subj2all['2023-SE2-024']
#vs= subj2all['2023-SE2-023']
#vs= subj2all['2023-SE2-002']
vs= subj2all[subj]
if 'lf' in vs:
    print('exc was')
    k = 'lf'            
    d['len_ts_di']     = len( vs[k]['ts_di']     )
    d['len_ts_di_off'] = len( vs[k]['ts_di_off'] )
elif  'loc_events2df' in vs:
    print('good')
    k = 'loc_events2df'
    d['len_ts_di']     = len( vs[k]['ts_di_']     )
    d['len_ts_di_off'] = len( vs[k]['ts_di_off_'] )
    
dat_time = vs['dat_time']
dfev = vs['dfev']
df = vs['df']

In [ ]:
(dfev['nsample'].diff()[1:] > 0).all()

In [ ]:
np.all(np.diff(dat_time[0,dfev['nsample']] )[1:])

In [ ]:

#################
c = dfev['phase'] != dfev['phase'].shift(1)
c[0] = False
dfev['phase_ind'] = np.cumsum(c)
dfevph = aggRows(dfev, 'time', 'max', colgrp = 'phase_ind')

print( (dfev['nsample'].diff()[1:] > 0).all())
#print( (dfev['time_megchan'].diff()[1:] > 0).all() )

In [ ]:
dfph.plot(y='time_since_first_REST')

In [ ]:
dfev.plot(y='time_since_first_REST')

In [ ]:
subj

In [ ]:
#ts_ev = vs[k]['ts_ev']
ts_ev = vs[k]['mvt_starts_megtrig']['time_since_first_REST'].values
ts_di = vs[k]['ts_di']
assert (np.diff( ts_ev ) > 0).all()
assert (np.diff( ts_di ) > 0).all()
len(ts_ev),len(ts_di)

In [ ]:
#ts_ev = vs[k]['ts_ev']
ts_ev = vs[k]['iti_starts_megtrig']['time_since_first_REST'].values
ts_di = vs[k]['ts_di_off']
assert (np.diff( ts_ev ) > 0).all()
assert (np.diff( ts_di ) > 0).all()
len(ts_ev),len(ts_di)

In [ ]:
from meg_proc import find_correspondence
    #return correspondences,paths

max_dist = 30 # Define your max distance threshold

#sl = slice(475,484)
#sl = slice(850,856)
sl = slice(None)

arr1 = dfev.query('phase == "REST"')['time_since_first_REST'].values
arr2 = dfph.query('phase == "REST"')['time_since_first_REST'].values

#arr1,arr2 = ts_ev[sl], ts_di[sl]
dfd,dfd_good,dfd_bad,paths = find_correspondence(arr1, arr2, max_dist)
best_path = dtw.best_path(paths)
print(len(dfd))
#print(corr_inds)

#dfd.query('dist > 1.')

In [ ]:
dfd['dist'].describe()

In [ ]:
dfd_bad = dfd[dfd['dist'] > dfd['dist'].max() * 0.95]
# ind1 is index in ts_ev
ind1 = dfd_bad['ind1'].values[0]
dfd_bad

In [ ]:
ti = vs[k]['iti_starts_megtrig'].reset_index().iloc[ind1]['trial_index']
ti = int(ti)
print(ti)

In [ ]:
dtwvis.plot_warping(arr1, arr2, best_path)

In [ ]:
fig = plt.figure(figsize=(6,6))
dtwvis.plot_warpingpaths(arr1, arr2, paths, best_path, figure =fig)

In [ ]:
paths.shape

In [ ]:
from meg_proc import read_final_rwd_info_MEG_CC

fnf = pjoin(path_behav,fnb+'.log')
print(path_behav)
read_final_rwd_info_MEG_CC(fnf)

In [ ]:
# Combine the phase columns to find all unique phases across both datasets
#all_phases = pd.concat([dfph['phase'], dfevph['phase']]).unique()
#all_phases = pd.concat([dfph_diext['phase'], dfevph_diext['phase']]).unique()
#display(all_phases)
all_phases = ['diode_OFF_to_ON','diode_ON_to_OFF',
              'TRAINING_START', 'REST',
       'GO_CUE_WAIT_AND_SHOW', 'TARGET_AND_FEEDBACK', 
       'ITI', 'TRAINING_END', 'BREAK', 'MEG_start_trigger_native', np.nan,
       'MEG_start_trigger']


# Assign a unique color and marker to each phase
colors = plt.cm.get_cmap('tab20', len(all_phases))
bms = ['<','>','^', 'v','o', 's', 
           'D',  '*', 'P', 'X','1','2']
markers = bms * (len(all_phases) // len(bms) + 1)
phase_color_marker = {phase: (colors(i), markers[i]) for i, phase in enumerate(all_phases)}

# Plot one dataset

In [ ]:
# trial-based
#time_since_prev_phase
cols = ['trial_index','phase_ind','phase','time_since_first_REST']#,'time']
ti1,ti2 = max(0,ti-2),ti+2

#ti1,ti2 = 149-2,149+2
#ti1,ti2 = 0,3
#dftmp1 = dfph[cols].query('trial_index <= @ti2 and trial_index >= @ti1')
print('trials ',ti1,ti2)
#dftmp2 = dfevph[cols].query('trial_index <= @ti2 and trial_index >= @ti1')
dftmp2 = dfph[cols].query('trial_index <= @ti2 and trial_index >= @ti1')
#[cols].iloc[sl],dfevph[cols].iloc[sl]

#from behav_proc import compDf
#compDf(dftmp1,dftmp2,cols, cols_to_comp=['trial_index','phase'])

In [ ]:
dftmp2

In [ ]:
# Visualize: Create a plot for each dataset
fig, axs = plt.subplots(3, 1, figsize=(15, 8), sharex=True)
fig.suptitle('Phase Occurrences Over Time')

dfdi = loc['dfdi']

# Plot for the first dataset
# for phase, (color, marker) in phase_color_marker.items():
#     subset = dftmp1[dftmp1['phase'] == phase]
#     if not subset.empty:
#         axs[0].scatter(subset['time_since_first_REST'], [phase]*len(subset), color=color, marker=marker, label=phase)

# Plot for the second dataset, using the adjusted time
for phase, (color, marker) in phase_color_marker.items():
    subset = dftmp2[dftmp2['phase'] == phase]
    if not subset.empty:
        axs[1].scatter(subset['time_since_first_REST'], [phase]*len(subset), color=color, marker=marker, label=phase)

    #df_diode[]

xlim = axs[0].get_xlim()
axs[2].plot(dfdi['time_since_first_REST'], dfdi['diode'])

#axs[0].plot(df['time_since_first_REST']  ,  df['error_distance'])
axs[0].plot(df['time_since_first_REST']  ,  df['jax1']**2 + df['jax2']**2)
#axs[0].plot(df['time_since_first_REST']  ,  df['jax2'])
    
#axs[0].vlines( dftmp1.loc[dftmp1['phase'] == 'ITI','time_since_first_REST'],0,4, ls=':')        
for ax in axs:
    a,b = ax.get_ylim()
    ax.vlines( dftmp2.loc[dftmp2['phase'] == 'TARGET_AND_FEEDBACK','time_since_first_REST'],a,b, ls=':',
             color = 'k')
    ax.vlines( dftmp2.loc[dftmp2['phase'] == 'ITI','time_since_first_REST'],a,b, ls=':',
             color = 'k')
        
axs[0].set_title('First Dataset (dfph)')
axs[0].legend(loc='upper left', bbox_to_anchor=(1, 1))
axs[0].set_ylabel('Phase')

axs[1].set_title('Second Dataset (dfevph)')
axs[1].legend(loc='upper left', bbox_to_anchor=(1, 1))
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Phase')

for ax in axs:
    #ax.set_xlim(100,117)
    ax.set_xlim(xlim)

plt.tight_layout(rect=[0, 0, 0.85, 0.95])
plt.show()

In [ ]:
dfev.query('trial_index >= 482 and trial_index <= 482')

In [ ]:
assert not dfev.duplicated('nsample').any()

In [ ]:
if params['subject'] == '2023-SE2-002':
    cti = dfev['nsample'] == 1184635
    dfev.loc[cti, 'trigger'] = 44 #ITI
dfev.loc[cti, 'tgti_to_show'] = 0 #ITI
dfev.loc[cti, 'phase'] = 'ITI
dfev.loc[cti, 'trial_info'] = ('perturbation', 'rot-20', 0, 'ITI')

# Two datasets

In [ ]:
tis

In [ ]:
# trial-based
#time_since_prev_phase
cols = ['trial_index','phase_ind','phase','time_since_first_REST']#,'time']
ti1,ti2 = max(0,ti-2),ti+2; print('ti = ',ti)


#ti1,ti2 = 0, 11
#ti1,ti2 = 0, 1
ti1,ti2 = 896, 900

#ti1,ti2 = 13,16
#ti1,ti2 = 147-2,148
#ti1,ti2 = 0,12
#ti1,ti2 = 0,3
#dftmp1 = dfph[cols].query('trial_index <= @ti2 and trial_index >= @ti1')
print('trials ',ti1,ti2)
qs = 'trial_index <= @ti2 and trial_index >= @ti1'
dftmp1 = dfev[cols].query(qs)
dftmp2 = dfph[cols].query(qs)
#[cols].iloc[sl],dfevph[cols].iloc[sl]

#from behav_proc import compDf
#compDf(dftmp1,dftmp2,cols, cols_to_comp=['trial_index','phase'])

In [ ]:
dfev['trial_index'].max(), dfph['trial_index'].max()

In [ ]:
subj

In [ ]:
stupid_align = 0
if subj == '2023-SE2-018' and stupid_align:
    # ALIGN second trial
    row = dfev.query('nsample == 5332') # GO_CUE_WAIT_AND_SHOW on second trial
    assert row['phase'].values[0] == 'GO_CUE_WAIT_AND_SHOW'
    t = row['time_since_first_REST'].values[0]

    row2 = dfph.query('phase == "GO_CUE_WAIT_AND_SHOW" and trial_index == 2')
    tbeh = row2['time_since_first_REST'].values[0]

    dif = t - tbeh
    print(dif)
    df.loc[df.query('time_since_first_REST >= @tbeh').index, 'time_since_first_REST'] += dif
    dfph.loc[dfph.query('time_since_first_REST >= @tbeh').index, 'time_since_first_REST'] += dif

    # ALIGN after TRAINING

    # in defev jump happens at nsample = 58007 (phase = MEG_start_trigger)
    #in dfev   REST after MEG_start_trigger
    # nsample = 60715
    row = dfev.query('nsample == 60715')
    assert row['phase'].values[0] == 'REST'
    t = row['time_since_first_REST'].values[0]

    row2 = dfph.query('phase == "REST" and phase.shift(1) == "TRAINING_END" and trial_index <= 14 and trial_index > 1')
    tbeh = row2['time_since_first_REST'].values[0]
    dif = t - tbeh
    df.loc[df.query('time_since_first_REST >= @tbeh').index, 'time_since_first_REST'] += dif
    dfph.loc[dfph.query('time_since_first_REST >= @tbeh').index, 'time_since_first_REST'] += dif
    print(t,tbeh)

### tmp

In [ ]:
ax = plt.gca()
t = 870
#t = 15
dfph.query('trial_index <= @t').plot(x='time_since_first_REST',y=['phase_ind'], ax = ax)
# event based is red
dfev.query('trial_index <= @t').plot(x='time_since_first_REST',y=['phase_ind'], ax = ax, color='red')

In [ ]:
dftmp = dfev.query(qs).copy()
dftmp['crest'] = loc['crest'].loc[dftmp.index]

dftmp[['trial_index','trigger','phase','prev_phase','crest',
       'phase_ind','time_since_first_REST','nsample']]

In [ ]:
dfph.query(qs)[['trial_index','phase','phase_ind','time_since_first_REST','trial_type','time','time_abs']]

###   f

In [ ]:
times_crest = dfev.loc[loc['ph'].loc[loc['cti']].index].query(qs)['time_since_first_REST']

# Visualize: Create a plot for each dataset
fig, axs = plt.subplots(4, 1, figsize=(15, 10), sharex=True)
fig.suptitle('Phase Occurrences Over Time')

dfdi = loc['dfdi']

##Plot for the first dataset
for phase, (color, marker) in phase_color_marker.items():
    subset = dftmp1[dftmp1['phase'] == phase]
    if not subset.empty:
        axs[0].scatter(subset['time_since_first_REST'], [phase]*len(subset), color=color, marker=marker, label=phase)

# Plot for the second dataset, using the adjusted time
for phase, (color, marker) in phase_color_marker.items():
    subset = dftmp2[dftmp2['phase'] == phase]
    if not subset.empty:
        axs[1].scatter(subset['time_since_first_REST'], [phase]*len(subset), color=color, marker=marker, label=phase)

    #df_diode[]

xlim = axs[0].get_xlim()
#axs[2].plot(vs[k]['dfdi']['time_since_first_REST'], vs[k]['dfdi']['diode'])

axs[2].plot(dfdi['time_since_first_REST'], dfdi['diode'])
axs[3].plot(df['time_since_first_REST']  ,  df['jax1']**2 + df['jax2']**2)

#axs[0].plot(df['time_since_first_REST']  ,  df['jax2'])
    
#axs[0].vlines( dftmp1.loc[dftmp1['phase'] == 'ITI','time_since_first_REST'],0,4, ls=':')        
for ax in axs:
    a,b = ax.get_ylim()
    a1,b1 = a, a+(b-a)/2
    a2,b2 = a+(b-a)/2,b
    ax.vlines( dftmp2.loc[dftmp2['phase'] == 'TARGET_AND_FEEDBACK','time_since_first_REST'],a1,b1, ls=':',
             color = 'k')
    ax.vlines( dftmp2.loc[dftmp2['phase'] == 'ITI','time_since_first_REST'],a1,b1, ls=':',
             color = 'k',label='dftmp2')
    
    ax.vlines( dftmp1.loc[dftmp1['phase'] == 'TARGET_AND_FEEDBACK','time_since_first_REST'],a2,b2, ls=':',
             color = 'r')
    ax.vlines( dftmp1.loc[dftmp1['phase'] == 'ITI','time_since_first_REST'],a2,b2, ls=':',
             color = 'r',label='dftmp1')
    
    ax.vlines( times_crest,a2,b2, ls='--',
             color = 'r')
    
        
axs[0].set_title('First Dataset (dfev)')
axs[0].legend(loc='upper left', bbox_to_anchor=(1, 1))
axs[0].set_ylabel('Phase')

axs[1].set_title('Second Dataset (dfph)')
axs[1].legend(loc='upper left', bbox_to_anchor=(1, 1))
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Phase')

for ax in axs:
    #ax.set_xlim(100,117)
    ax.set_xlim(xlim)

plt.tight_layout(rect=[0, 0, 0.85, 0.95])
plt.show()

In [ ]:
#018 on second trial in dfph missing post-REST phases. 
# looks like there was almost no cursor mvt but the diode turn on and off

In [ ]:
dfph.query('trial_index <= 4')[cols]

In [ ]:
p = '/home/demitau/data_Quentin/full_experiments/data2/prl_'
for ind in range(2):
    fnf = p + str(ind)  + '.npz'
    print(fnf)
    f = np.load(fnf, allow_pickle=True)
    ff = f['arr_0']
    

    prl2 = []
    for i in range(len(ff)):
        summ = ff[i]['res'].summary() #remove_data()
        prl2 += [summ]


    np.savez(fnf,prl2)

In [ ]:
    for i in range(len(ff)):
        try:
            ff[i]['res'].remove_data()
        except AttributeError as e:
            print(i,'not')


In [ ]:
res.remove_data?

In [ ]:
res._results.model.data = None
res._results.model.row_indices = None
res._results.model.endog_li = None
res._results.model.exog_li = None
res._results.model.exog_re_li = None
res._results.model._quad = None
res._results.model._endex_li = None
res._results.model._aex_r = None

In [ ]:
res.summary()

In [ ]:
display(vars(vs['_results'].model ))

In [ ]:
vs.keys()